# Capstone Project - The Battle of the Neighborhoods (Week 2)


## Table of contents

* Introduction: Problem definition
* Data
* Methodology
* Analysis
* Results and Discussion
* Conclusion

# Introduction:Problem definition  

Aldo is 35 years old and moved from Milan to Toronto. He has 15 years of experience as Italian chef and now wishes to get started with his restaurant in Toronto. Since he is not familiar with the city, we will try to help him with some data to decide which is the appropriate neighbourhood for his activity.  

Among other things, Aldo need to know:
*	the level of competition on each neighbourhood 
*	which are the most dynamic neighbourhoods
*	have an idea of Toronto neighbourhoods


# Data

Based on our business problem we are interested to identify and evaluate the following parameters: 

* number of Italian restaurants on each neighbourhood 
* number of hotels on each neighbourhood, as people who move for pleasure or business have high likelihood to be client segment 
* ten most common activities for each neighbourhood 

Toronto neighbourhoods are defined by latitude and longitude that we upload in our notebook by an external file, it is publicly available for download. 

Number and category of venues for each neighbourhood are obtained using Foursquare API. 


#### Import Canada file and geospatial coordinates file in Python

Import Canada file (it is an excel file) in Python and read it in pandas dataframe 

In [1]:
# The code was removed by Watson Studio for sharing.

We start to work on this file with some simple operations: rename the column Postal code, remove non assigned values from column Borough and check how many unique values are in the column PostalCode


In [2]:
#rename the column to PostalCode
df.rename(columns={'Postal code':'PostalCode'}, inplace=True)

# create a new dataframe without 'Not assigned' values
df_data = df[df['Borough'] != 'Not assigned'].reset_index(drop=True)

# check how many unique values are in the column 'PostalCode'
len(df_data['PostalCode'].unique())

103

In [3]:
# At ths point we check if the uniques values are equal to the total number of rows in df_data

df_data.shape

(103, 3)

Clean the column Neighborhood and check if it has Not assigned values

In [4]:
# replace frontslash with comma 
df_data['Neighborhood'] = df_data.Neighborhood.str.replace('/',',')

# check if 'Not assigned' value is present in 'Neighborhood' column
found = df_data[df_data['Neighborhood'].str.contains('Not assigned')]
print(found.count())

PostalCode      0
Borough         0
Neighborhood    0
dtype: int64


Import Geospatial Coordinates file in Python, it is a csv file

In [5]:
body = client_0300fc5a45584f96bdd6e5337a34cb86.get_object(Bucket='applieddatasciencecapstone-donotdelete-pr-xgqhdbkfmjrxtt',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_coordinates = pd.read_csv(body)
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We need to merge df_coordinates and df_data. For this reason we rename the column to PostalCode and check the dataframe size

In [6]:
#rename the column as it is in df_data 
df_coordinates.rename(columns={'Postal Code':'PostalCode'}, inplace=True)

# check the size of the dataframe 
df_coordinates.shape

(103, 3)

In [7]:
# df_data and df_coordinates have in comon 'PostalCode' column, this help us to merge the two dataframes 

df_merged = df_data.merge(df_coordinates, on=['PostalCode'], how='inner')
df_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


In [8]:
df_merged.shape

(103, 5)

We are interested to investigate only Toronto city; therefore, we choose Toronto in Borough column

In [9]:
toronto_data = df_merged[df_merged['Borough'].str.contains("Toronto")].reset_index(drop=True)

# clean data in column 'Borough' ... we are interested to have only Toronto
toronto_data.Borough = [col.split()[-1] for col in toronto_data.Borough]

# look at the dataframe
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
1,M7A,Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
2,M5B,Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Toronto,St. James Town,43.651494,-79.375418
4,M4E,Toronto,The Beaches,43.676357,-79.293031


In [10]:
# check the presenze of a unique value in 'Borough' column
len(toronto_data['Borough'].unique())

1

#### Investigate higher level and direct level of competition on each neighbourhood 


we start with importing the necessary libraries 

In [11]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

Libraries imported.


In [12]:
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library


From the lab we call the function getNearbyVenues

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### In order to use the function we need to define first our credientials plus version, limit and radius variables

In [14]:
# The code was removed by Watson Studio for sharing.

In [15]:
# apply getNearbyVenues function and get toronto_venues

toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park , Harbourfront
Queen's Park , Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond , Adelaide , King
Dufferin , Dovercourt Village
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
The Danforth West , Riverdale
Toronto Dominion Centre , Design Exchange
Brockton , Parkdale Village , Exhibition Place
India Bazaar , The Beaches West
Commerce Court , Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park , The Junction South
North Toronto West
The Annex , North Midtown , Yorkville
Parkdale , Roncesvalles
Davisville
University of Toronto , Harbord
Runnymede , Swansea
Moore Park , Summerhill East
Kensington Market , Chinatown , Grange Park
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Roseda

In [16]:
# check df toronto_venues dimensions
toronto_venues.shape

(1623, 7)

In [17]:
# look at the dataframe toronto_venues
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park , Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park , Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park , Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park , Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park , Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


 check the number of unique categorie 

print('There are {} uniques categories'.format(len(toronto_venues['Venue Category'].unique())))

# Methodology

In this project we are focused on identifying the most common venues for each neighbourhood of Toronto, in this way we get some more confidence with a city. 

In the second step we filter the data and identify the total number of restaurants present in Toronto, this help us to identify the higher level of competition. Later, we go in more detail and identify the number of restaurants that offer a Mediterranean and European cuisine. We and this step with the identification of Italian restaurants and the identification of hotels in Toronto. 

In the final step we use folium to create a map of Toronto city; we also use plugins function to group markers into different clusters, this make easier identification of areas with higher density of venues. 

In this project we use different defined function, very useful and necessary to complete the analysis. More details of this functions can be found in the attached notebook. 


# Analysis

#### Know we start looking at the broad level of competition, so we check how many restaurants are present in Toronto. At this point we do not investigate in detail the restaurant offers

In [19]:
toronto_restaurant = toronto_venues[toronto_venues['Venue Category'].str.contains('Restaurant')].reset_index(drop=True)
print('There are {} restaurants in Toronto'.format(len(toronto_restaurant)))

There are 387 restaurants in Toronto


#### Knowing the higher level of competition, we precede to investigate in more detail the Mediterranean and European cuisine

In [20]:
toronto_restaurant = toronto_venues[toronto_venues['Venue Category'].str.contains\
                                    ('Mediterranean Restaurant|Greek Restaurant|Italian Restaurant|French Restaurant')].reset_index(drop=True)
toronto_restaurant.head()


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park , Harbourfront",43.654260,-79.360636,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant
1,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,Mercatto,43.660391,-79.387664,Italian Restaurant
2,"Garden District, Ryerson",43.657162,-79.378937,Scaddabush Italian Kitchen & Bar,43.658920,-79.382891,Italian Restaurant
3,"Garden District, Ryerson",43.657162,-79.378937,Trattoria Mercatto,43.654453,-79.380974,Italian Restaurant
4,"Garden District, Ryerson",43.657162,-79.378937,Donatello Restaurant,43.657489,-79.383605,Italian Restaurant


In [21]:
# check the size of the dataframe 
print('There are {} Mediterranean and European restaurants in Toronto'.format(len(toronto_restaurant)))

There are 69 Mediterranean and European restaurants in Toronto


#### There is one more venue we are interested in, the number of hotels present in Toronto. We add this new venue to our dataframe and store the new data in selected_venues 

In [22]:
selected_venues = toronto_venues[toronto_venues['Venue Category'].str.contains\
                                    ('Mediterranean Restaurant|Greek Restaurant|Italian Restaurant|French Restaurant|Hotel')].reset_index(drop=True)
selected_venues.shape

(103, 7)

In [23]:
# check how many venues are returned for each neighborhood

selected_venues.groupby('Venue Category').count().sort_values('Neighborhood', ascending = False)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Italian Restaurant,41,41,41,41,41,41
Hotel,32,32,32,32,32,32
Greek Restaurant,14,14,14,14,14,14
French Restaurant,10,10,10,10,10,10
Mediterranean Restaurant,4,4,4,4,4,4
Hotel Bar,2,2,2,2,2,2


As we can see in the above table, on 69 Mediterranean and European restaurants 41 are Italian restaurants, 60 percent

#### Know we want to analyse the number of the selected venues on each neighbourhood

In [24]:
# get dummy variables on column 'Venues Category'
venues_dummy = pd.get_dummies(selected_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
venues_dummy['Neighborhood'] = selected_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [venues_dummy.columns[-1]] + list(venues_dummy.columns[:-1])
venues_dummy = venues_dummy[fixed_columns]
venues_dummy.head()

,Neighborhood,French Restaurant,Greek Restaurant,Hotel,Hotel Bar,Italian Restaurant,Mediterranean Restaurant
0,"Regent Park , Harbourfront",1,0,0,0,0,0
1,"Regent Park , Harbourfront",0,0,1,0,0,0
2,"Queen's Park , Ontario Provincial Government",0,0,0,0,1,0
3,"Garden District, Ryerson",0,0,1,0,0,0
4,"Garden District, Ryerson",0,0,0,0,1,0


#### Know we group rows by Neighborhood and take the sum of each category 

In [25]:
# we want to know the number of restaurants and hotels for each Neighborhoods 
venues_grouped = venues_dummy.groupby('Neighborhood').sum().sort_values('Italian Restaurant', ascending = False).reset_index()
venues_grouped.head()

,Neighborhood,French Restaurant,Greek Restaurant,Hotel,Hotel Bar,Italian Restaurant,Mediterranean Restaurant
0,Central Bay Street,1,0,1,0,4,0
1,"Harbourfront East , Union Station , Toronto Is...",0,0,4,1,3,0
2,"Garden District, Ryerson",0,0,2,0,3,0
3,"Toronto Dominion Centre , Design Exchange",1,1,6,0,3,0
4,"Commerce Court , Victoria Hotel",1,0,5,0,3,0


In [26]:
venues_grouped.shape

(27, 7)

#### Create the map of Toronto using folium. First we will find Toronto latitude and longitude and then will create the map

In [27]:
# find lat and lon of Tornoto
address = 'Toronto, CA'
geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Tornoto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Tornoto are 43.6534817, -79.3839347.


In [28]:
# create map of Tornot using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], 
                                           toronto_data['Longitude'], 
                                           toronto_data['Borough'], 
                                           toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  





##### On the map we have created we are interested to show the venues for each neighbourhood. So, we create a feature group for the venues in the dataframe and loop through venues to add them to the feature group. 





In [29]:
# instantiate a feature group for the venues in the dataframe
venues = folium.map.FeatureGroup()

# loop through venues and add each to the feature group
for lat, lng, in zip(toronto_restaurant['Venue Latitude'], toronto_restaurant['Venue Longitude']):
    venues.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, 
            color='yellow',
            fill=True,
            fill_color='red',
            fill_opacity=0.6
        )
    )
    
# add pop-up text to each marker on the map
latitudes = list(toronto_restaurant['Venue Latitude'])
longitudes = list(toronto_restaurant['Venue Longitude'])
labels = list(toronto_restaurant['Venue Category'])

for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label).add_to(map_toronto)

# add incidents to map
map_toronto.add_child(venues)

##### As it is hard to read the map we have done, we preced by grouping the markers into different clusters. Each cluster is then rapresented by the number of venues in each neighborhood. By zooming the global cluster will start breaking up into smaller clusters

In [30]:
from folium import plugins

# let's start with a clean copy of the map of Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# instantiate a mark cluster object for the incidents in the dataframe
venues = plugins.MarkerCluster().add_to(map_toronto)

# loop through venues and add each to the feature group
for lat, lng, label in zip(toronto_restaurant['Venue Latitude'], toronto_restaurant['Venue Longitude'], toronto_restaurant['Venue Category']):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(venues)
    
# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], 
                                           toronto_data['Longitude'], 
                                           toronto_data['Borough'], 
                                           toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Investigate the most common venues for each neighbourhood 

For this objective we create dummy variables on toronto_venues dataframe, in the column ‘Venue Category’. We will store the data in toronto_dummy dataframe. After that we will group the data and use the function: return_most_common_venues … we will import it from the lab. 

In [31]:
# get dummy variables on column 'Venues Category'
toronto_dummy = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_dummy['Neighborhood'] = toronto_venues['Neighborhood']
toronto_dummy.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Neighborhood is not the last column of the dataframe, we need to find its position and bring it as first column.
In order to do this we first transform columns into a list, find the position of our column and then move it as
first column in the dataframe

In [43]:
# trasform columns of toronto_dummy dataframe into a list
cols = toronto_dummy.columns.tolist()

# check the position of 'Neighborhood' column into the list
print(cols.index('Neighborhood'))

74


In [44]:
# move neighborhood column to the first column
cols = cols[74:] + cols[:74]

toronto_dummy = toronto_dummy[cols]
toronto_dummy.head()

,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue
0,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0

In [45]:
# look at the dimensions of the dataframe toronto_dummy
toronto_dummy.shape

(1623, 242)

#### Let's create a new dataframe where data are grouped by column 'Nighborhood' and let' use the mean function to see the frequency distribution


In [46]:
toronto_grouped = toronto_dummy.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue
0,Berczy Park,0.000000,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.018182,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.018182,0.000000,0.000000,0.000000,0.000000,0.036364,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.036364,0.000000,0.000000,0.018182,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018182,0.0000,0.000000,0.000,0.018182,0.000000,0.0000,0.000000,0.000000,0.00,0.018182,0.000000,0.00,0.000000,0.018182,0.000000,0.000000,0.000000,0.00

#### Import from the lab most_common_venues function and the process to store the data in dataframe neighborhoods_venues_stored 

In [47]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [48]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Café,Bakery,Restaurant,Cheese Shop,Beer Bar,Seafood Restaurant,Comfort Food Restaurant,BBQ Joint
1,"Brockton , Parkdale Village , Exhibition Place",Café,Breakfast Spot,Coffee Shop,Burrito Place,Climbing Gym,Italian Restaurant,Stadium,Restaurant,Intersection,Convenience Store
2,Business reply mail Processing CentrE,Light Rail Station,Garden,Pizza Place,Skate Park,Smoke Shop,Auto Workshop,Restaurant,Spa,Recording Studio,Comic Shop
3,"CN Tower , King and Spadina , Railway Lands , ...",Airport Terminal,Airport Service,Airport Lounge,Bar,Airport Gate,Airport,Sculpture Garden,Plane,Boat or Ferry,Airport Food Court
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Ice Cream Shop,Restaurant,Japanese Restaurant,Bar,Salad Place,Burger Joint
5,Christie,Grocery Store,Café,Park,Restaurant,Diner,Coffee Shop,Italian Restaurant,Candy Store,Baby Store,Athletics & Sports
6,Church and Wellesley,Sushi Restaurant,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Mediterranean Restaurant,Café,Bubble Tea Shop,Hotel,Pub
7,"Commerce Court , Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Japanese Restaurant,Deli / Bodega,Italian Restaurant,Seafood Restaurant
8,Davisville,Dessert Shop,Sandwich Place,Pizza Place,Coffee Shop,Sushi Restaurant,Gym,Italian Restaurant,Café,Seafood Restaurant,Thai Restaurant
9,Davisville North,Gym,Food & Drink Shop,Sandwich Place,Hotel,Dog Run,Convenience Store,Park,Department Store,Breakfast Spot,Video Store


# Results and discussion 

At the higher level of competition, we have found that 387 restaurants are present in Toronto. Making a more detail analysis we have seen that 69 of them offer a Mediterranean and European cuisine. The number of Italian restaurants in Toronto is 41. Furthermore, we have found that are 34 hotel presents in Toronto. 

The area between University of Toronto – St James Town – Berczy Park has the higher density of venues, there are 41 venues. The second most dense are is located between The Danforth West – Studio District, there are 13 venues.   The third most dense area is located between High Park – Runnymede – Parkdale, there are 5 venues. 

Central Bay Street and The Danforth West are the neighbourhoods with high level of competition, they have Italian Restaurant as 2nd most common venue. At the same time Danforth West has a high level of Greek restaurants, they are the 1st most common venue of the neighbourhood. 
 
There are 20 neighbourhood that do not have an Italian restaurant among ten most common venues. 

# Conclusions

In our opinion neighbourhoods Studio District; Richmond, Adelaide, King; Davisville North are three dynamic areas and do not have Italian restaurants among the 10 common venues. They can be a good starting point to evaluate the placement of the new restaurant.  

Our work is focused in analysis of the data, before the final decision is necessary to make an economic and financial analysis, all aspect that are outside the scope of this report. 
